<a href="https://colab.research.google.com/github/hyoh79/ML/blob/main/SL(cancer_classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer=load_breast_cancer()
print("cancer.keys():\n",cancer.keys())

In [ ]:
print("유방암 데이터의 형태:",cancer.data.shape)
print("유방암 데이터의 형태(또 다른 명령어 표현)", cancer['data'].shape)
print("유방암 데이터의 타입:", type(cancer)) # load_breat_cancer( )가 반환한 cancer 객체는 파이썬 딕셔너리와 유사한 Bunch 클래스의 객체로 키(keys)와 값(value)으로 구성되어 있기 때문

In [ ]:
import numpy as np
print("클래스별 샘플 개수:\n",{n:v for n,v in zip(cancer.target_names,np.bincount(cancer.target))})

In [ ]:
print("특성 이름:\n",cancer.feature_names)

In [ ]:
print("데이터 정보:\n",cancer.DESCR)

In [ ]:
print(cancer.data.shape)
print(cancer['feature_names'])
print(cancer['data'][:5])

In [ ]:
print(cancer['target'])
print(type(cancer['target']))
print(cancer.target.shape)

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, stratify=cancer.target, random_state=66)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancer['data'],cancer['target'],stratify=cancer.target,random_state=66)

In [ ]:
import pandas as pd
!pip install mglearn

import mglearn

cancer_dataframe=pd.DataFrame(X_train, columns=cancer.feature_names)
pd.plotting.scatter_matrix(cancer_dataframe, c=y_train, figsize=(15,15),marker='o',hist_kwds={'bins':20},s=60,alpha=.8,cmap=mglearn.cm3)

In [ ]:
training_accuracy=[]
test_accuracy=[]

neighbors_settings=range(1,11) #1에서 10까지 n_neighbors를 적용

## [Algorithm#1: kNN for classification]
from sklearn.neighbors import KNeighborsClassifier
for n_neighbors in neighbors_settings:
  clf=KNeighborsClassifier(n_neighbors=n_neighbors)
  clf.fit(X_train, y_train)
  training_accuracy.append(clf.score(X_train,y_train))
  test_accuracy.append(clf.score(X_test,y_test))

In [ ]:
import matplotlib.pylab as plt
plt.plot(neighbors_settings,training_accuracy,label="Trainig Accuracy")
plt.plot(neighbors_settings, test_accuracy, label="Test Accuracy")
plt.ylabel("accuracy")
plt.xlabel("# of neighbors")
plt.legend()
# 이웃을 하나 선택했을때 결정경계가 훈련 데이터에 가깝게 따라감(복잡한 모델, 과대적합). 
# 이웃의 수를 늘릴수록 결정 경계는 더 부드러워짐(단순한 모델을 의미).
# cancer dataset으로 kNN에서 이웃의 수를 변화시켜보면서 모델의 복잡도와 일반화 사이의 관계를 입증해볼 수 있음.

In [ ]:
## [Algorithm#2: Logistic Regression for classification]

from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(max_iter=5000).fit(X_train,y_train)

print("훈련 세트 점수: {:.3f}".format(logreg.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(logreg.score(X_test,y_test)))
# 기본값 C=1이 훈련세트와 테스트 세트 양쪽에 95% 정확도로 꽤 훌륭한 성능을 내고 있지만
# 훈련세트와 테스트 세트의 성능이 매우 비슷 (과소적합)--> 해결책(모델의 regularization-제약/규제을 더 풀어주기 위해 C를 증가시켜볼것!)

In [ ]:

logreg100=LogisticRegression(C=100,max_iter=50000).fit(X_train,y_train) # C를 증가시킴(복잡도를 높인 모델, 성능이 좋음)

print("훈련 세트 점수: {:.3f}".format(logreg100.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(logreg100.score(X_test,y_test)))

In [ ]:

logreg001=LogisticRegression(C=0.01,max_iter=50000).fit(X_train,y_train) 
# C를 감소시킴.(regularization-규제를 높인 모델, 이미 과소적합된 모델에서 규제를 더 높였기에 기본 매개변수 C=1일때보다 성능이 낮아짐)

print("훈련 세트 점수: {:.3f}".format(logreg001.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(logreg001.score(X_test,y_test)))

In [ ]:
# 유방암 데이터셋에 각기 다른 C값을 사용하여 만든 로지스틱 회귀의 계수 (예: L2규제)
import matplotlib.pyplot as plt
plt.plot(logreg100.coef_.T, '^', label="C=100")
plt.plot(logreg.coef_.T,'o',label="C=1")
plt.plot(logreg001.coef_.T,'v',label="C=0.001")

plt.xticks(range(cancer.data.shape[1]),cancer.feature_names, rotation=90)

xlims=plt.xlim()
plt.hlines(0,xlims[0],xlims[1])
plt.xlim(xlims)
plt.ylim(-5,5)

plt.ylabel("Coefficients of the trained model")
plt.xlabel("Features")

plt.legend()

In [ ]:
# 유방암 데이터셋에 각기 다른 C값을 사용하여 만든 로지스틱 회귀의 계수 (예: L1규제)
for C, marker in zip([0.001,1,100],['o','^','v']):
  lr_l1=LogisticRegression(solver='liblinear',C=C,penalty="l1",max_iter=1000).fit(X_train,y_train)
  print("C={:.3f}인 로지스틱 회귀의 훈련 정확도:{:.2f}".format(C,lr_l1.score(X_train,y_train)))
  print("C={:.3f}인 로지스틱 회귀의 훈련 정확도:{:.2f}".format(C,lr_l1.score(X_test,y_test)))
  plt.plot(lr_l1.coef_.T, marker, label="C={:.3f}".format(C))

  plt.xticks(range(cancer.data.shape[1]),cancer.feature_names,rotation=90)
  xlims=plt.xlim()
  plt.hlines(0,xlims[0],xlims[1])
  plt.xlim(xlims)
  plt.xlabel("Features")
  plt.ylabel("Coefficients of the trained model")

  plt.ylim(-5,5)
  plt.legend(loc=3)


In [ ]:
## [Algorithm#3: (kernel) SVC for classification]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(cancer.data,cancer.target,random_state=0)

from sklearn.svm import SVC
svc= SVC()
svc.fit(X_train,y_train)

print("훈련 세트 정확도: {:.2f}".format(svc.score(X_train,y_train)))
print("테스트 세트 정확도:{:.2f}".format(svc.score(X_test,y_test)))


In [ ]:
import matplotlib.pyplot as plt
plt.boxplot(X_train, manage_ticks=False)
plt.yscale("symlog")
plt.xlabel("Feature list")
plt.ylabel("Feature size")

In [ ]:
# 훈련 세트에서 특성별 최솟값 계산
min_on_training=X_train.min(axis=0)

# 훈련 세트에서 특성별 (최댓값-최솟값) 범위 계산
range_on_training=(X_train-min_on_training).max(axis=0)


# 훈련 데이터에서 최솟값을 빼고 범위로 나누면
# 각 특성에 대해 최솟값은 0 최댓값은 1입니다
X_train_scaled=(X_train-min_on_training) / range_on_training
print("특성별 최솟값\n", X_train_scaled.min(axis=0))
print("특성별 최댓값\n", X_train_scaled.max(axis=0))

# 테스트 세트에도 같은 작업을 적용하지만
# 훈련 세트에서 계산한 최솟값과 범위를 사용해야함. (Chap.3)
X_test_scaled=(X_test-min_on_training)/range_on_training

svc=SVC()
svc.fit(X_train_scaled, y_train)

print("훈련 세트 정확도: {:.3f}".format(svc.score(X_train_scaled,y_train)))
print("테스트 세트 정확도: {:.3f}".format(svc.score(X_test_scaled,y_test)))

In [ ]:
# C나 gamma값을 증가시켜 좀 더 복잡한 모델을 만들 수 있음.
svc=SVC(C=1000)
svc.fit(X_train_scaled, y_train)

print("훈련 세트 정확도: {:.3f}".format(svc.score(X_train_scaled,y_train)))
print("테스트 세트 정확도: {:.3f}".format(svc.score(X_test_scaled,y_test)))

In [ ]:
# C나 gamma값을 감소시켜 좀 더 규제를 강하게 함.
svc=SVC(C=000.1)
svc.fit(X_train_scaled, y_train)

print("훈련 세트 정확도: {:.3f}".format(svc.score(X_train_scaled,y_train)))
print("테스트 세트 정확도: {:.3f}".format(svc.score(X_test_scaled,y_test)))

In [ ]:
## [Algorithm#4: Decision Tree for classification]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree=DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)

print("훈련 세트 정확도: {:.3f}",format(tree.score(X_train,y_train)))
print("테스트 세트 정확도: {:.3f}",format(tree.score(X_test,y_test)))

In [ ]:
tree=DecisionTreeClassifier(max_depth=3,random_state=0)
tree.fit(X_train,y_train)

print("훈련 세트 정확도: {:.3f}", format(tree.score(X_train,y_train)))
print("테스트 세트 정확도: {:3f}", format(tree.score(X_test,y_test)))


In [ ]:
from sklearn.tree import export_graphviz
export_graphviz(tree,out_file="tree.dot",class_names=["악성","양성"], feature_names=cancer.feature_names, impurity=False,filled=True)

In [ ]:
import graphviz
with open("tree.dot") as f:
  dot_graph=f.read()
display(graphviz.Source(dot_graph))

In [ ]:
print("특성 중요도\n", tree.feature_importances_)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def plot_feature_importances_cancer(model):
  n_features=cancer.data.shape[1]
  print("n_features:",n_features)
  plt.barh(np.arange(n_features),model.feature_importances_,align='center')
  print("model.feature_importances_:",model.feature_importances_)
  plt.yticks(np.arange(n_features),cancer.feature_names)
  plt.xlabel("feature_importances")
  plt.ylabel("feature")
  plt.ylim(-1,n_features)

plot_feature_importances_cancer(tree)

In [ ]:
# !pip install mglearn

# import mglearn

tree=mglearn.plots.plot_tree_not_monotone()
display(tree)

In [ ]:
## [Algorithm#5: Random Forest for classification]
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=100,random_state=0)
forest.fit(X_train,y_train)


In [ ]:
# Random Forest는 아무런 매개변수튜닝없이도 linear model, Decision Tree보다 높은 정확도를 냄.
# 기본적으로 좋은 성능을 냄
print("훈련 세트 정확도:{:.3f}".format(forest.score(X_train,y_train)))
print("테스트 세트 정확도:{:.3f}".format(forest.score(X_test,y_test)))


In [ ]:
# 유방암 데이터로 만든 Random Forest Model의 특성 중요도 (각 트리의 특성 중요도를 취합하여 계산한것)
# Random Forest에서 제공하는 특성 중요도가 하나의 트리에서 제공하는 것보다 신뢰도가 높음.
# Radom은 알고리즘이 가능성 있는 많은 경우를 고려할 수 있도록 하므로 Random Forest가 단일 트리보다 더 넓은 시각으로 데이터를 바라볼 수 있음.
plot_feature_importances_cancer(forest)